<h1> Musikjahr </h1>

This Data Science project aims to analyze a Spotify user's music trends in the previous year and recommends new artists based on those music trends.

Install libraries related to API

In [1]:
%pip install python-dotenv
%pip install requests

Install libraries related to Data Science tasks

In [2]:
%pip install seaborn

Install library related to Spotify OAuth2

In [3]:
from dotenv import load_dotenv
import os
import base64
import requests
import urllib.parse
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer

Matplotlib is building the font cache; this may take a moment.


In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = "http://localhost:3001"
scope = "user-top-read"

In [6]:
# Returns the authorization header required to query the API
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

In [7]:
def get_auth_code():
    auth_url = "https://accounts.spotify.com/authorize"
    
    params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": "user-top-read",
    }

    auth_url = f"{auth_url}?{urllib.parse.urlencode(params)}"
    print(f"Please Authorize access by clicking on: {auth_url}")

    auth_code = input("Enter the code in the generated URL: ")
    
    return auth_code


In [8]:
# Returns new refresh token
def exchange_auth_code_for_refresh_token(auth_code):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": redirect_uri,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    refresh_token = json_result['refresh_token']
    
    return refresh_token

In [9]:
# Returns a new access token upon every execution
def refresh_access_token(refresh_token):
    token_url = "https://accounts.spotify.com/api/token"
    
    data = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "client_id": client_id,
        "client_secret": client_secret
        }

    result = requests.post(token_url, data=data)
    json_result = json.loads(result.content)
    new_access_token = json_result['access_token']

    return new_access_token

In [10]:
auth_code = get_auth_code()
print(auth_code)

Please Authorize access by clicking on: https://accounts.spotify.com/authorize?client_id=9e3ab84ec3634133b2710bc672715e09&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3001&scope=user-top-read
<PyodideFuture pending cb=[WebLoop._decrement_in_progress()]>


Enter the code in the generated URL:  AQAjHabII9gSdKCksDeMU49flKHSpR5bLAKdqfoFRrO2vs8Myl-dunG4mj5KLb4Jze6msCyiprYD99z6Is3_GolVFTCkArgLHHVJh18v4cZt6uoYsYnYenS6DjVg4yFmuGbvN_cDxTsQ-84Aq8Ob47guguha__qGFaTPOww8Jg3U2ofcxPrbbA


In [14]:
refresh_token = exchange_auth_code_for_refresh_token(auth_code.result())

In [54]:
api_url = "https://api.spotify.com/v1/me/top/tracks"

params = {
    "limit": 50,
    "time_range": "long_term",
    "offset": 0
}

result = requests.get(api_url, headers=get_auth_header(refresh_access_token(refresh_token)), params=params)
data = json.loads(result.content)

count=1

for song in data['items']:
    for artist in song['album']['artists']:
        print(str(count) + ". " + song['name'] + " - " + artist['name'])
        count+=1

1. Bothmer Schloss - Marie Bothmer
2. baby pink (feat. Eem Triplin) - Camila Cabello
3. 3 Nights - Dominic Fike
4. Feel It - From The Original Series “Invincible” - d4vd
5. Young And Beautiful - Various Artists
6. Don't Go Yet - Camila Cabello
7. Jailhouse Rock - Elvis Presley
8. Say Yes To Heaven - Lana Del Rey
9. On An Evening In Roma - Dean Martin
10. Sprinter - Dave
11. Sprinter - Central Cee
12. Paint The Town Blue (from the series Arcane League of Legends) - Ashnikko
13. Paint The Town Blue (from the series Arcane League of Legends) - Arcane
14. Paint The Town Blue (from the series Arcane League of Legends) - League of Legends
15. Vienna - Billy Joel
16. Celia - Camila Cabello
17. letzte bahn - LUNA
18. Not Another Rockstar - Maisie Peters
19. Mona Lisa (Spider-Man: Across the Spider-Verse) - Dominic Fike
20. Wildberry Lillet - Nina Chuba
21. Pope Is a Rockstar - SALES
22. Runaway - AURORA
23. Hello Mate - ArrDee
24. Hello Mate - Kyla
25. verlierer - LUNA
26. Indecisive - Is It A